If Dockerfiles have not been modified, connect to the Jupyter server with ```http://localhost:8002/tree?token=find-site-panos```  

This notebook describes a pipeline to find all Google Street View panoramas with an available depth map within a bounding rectangle.  
The bounding rectangle is specified by providing two [EPSG:4326](https://epsg.io/4326) coordinates that are used as corner points.  
Some code and packages are derived from [Street View DEMs](https://github.com/ccheemeng/streetview-dem).  

For each valid found panorama, its ID (serving as a primary key), latitude, and longitude are written to a ```.csv``` file.  
This ```.csv``` file may be used as an input for [```collect-street-view```](collect-street-view.ipynb).  

If necessary, the output ```.csv``` file may be preprocessed with a GIS software like [QGIS](https://www.qgis.org) to further refine the found panoramas beyond being defined by a rectangle.

In [ ]:
target_dir = "data"
lat1 = 1.3840874919650237
lon1 = 103.73928736838847
lat2 = 1.4066928731913269
lon2 = 103.76019645180367
output_filename = "yew-tee-locationss.csv"

In [2]:
import aiocsv
import aiofiles
from aiohttp import ClientSession, ClientTimeout

import geo
import streetview

import asyncio
import csv
from pathlib import Path
import os

In [3]:
async def main(p1, p2, resolution, target_crs, filepath):
    Path(filepath).parent.mkdir(parents=True, exist_ok=True)
    with open(filepath, 'w', newline='') as csvfile:
        csv_writer = csv.writer(csvfile)
        csv_writer.writerow(["id", "lat", "lon"])
    sample = geo.Sample(p1, p2, resolution)
    latlon = list(sample.generate_latlon_samples())
    lat = [x[0] for x in latlon]
    lon = [x[1] for x in latlon]
    radius = sample.search_radius()
    await write_panos(lat, lon, radius, target_crs, filepath)

async def write_panos(lat, lon, radius, target_crs, filepath):
    ids = set()
    async with ClientSession(timeout=ClientTimeout(total=None), raise_for_status=True) as session:
        async with aiofiles.open(filepath, 'w+') as afp:
            writer = aiocsv.AsyncWriter(afp)
            await writer.writerow(["id", "lat", "lon"])
            for i in range(len(lat)):
                async for row in get_latlon(session, ids, lat[i], lon[i], radius, target_crs):
                    await writer.writerow(row)

async def get_latlon(session, ids, lat, lon, radius, target_crs):
    pano = await streetview.StreetViewAPI.find_pano_full(session, lat, lon, radius)
    if pano and pano.id not in ids and pano.elevation and pano.depth:
        ids.add(pano.id)
        yield (pano.id, pano.lat, pano.lon)

In [4]:
await main((lat1, lon1), (lat2, lon2), 10, 4326, os.path.join(target_dir, output_filename))

cmHafmt9Bn0h0EKGPPLPig: invalid literal for int() with base 2: ''
9NIWaw61SMfms-68tWjdsw: invalid literal for int() with base 2: ''
TZxyrX9XOnbAGv-t2GZ6dw: invalid literal for int() with base 2: ''
3cTDqT6N6o-3i7N6TY1uLg: invalid literal for int() with base 2: ''
TZxyrX9XOnbAGv-t2GZ6dw: invalid literal for int() with base 2: ''
1Ctmg5mUcI2Kb7TsNOe9Hg: invalid literal for int() with base 2: ''
U2A7MYsquNhLQN78ZCtBXA: invalid literal for int() with base 2: ''
tyfMjR6f6vvkM6POJ-vUQQ: invalid literal for int() with base 2: ''
4_mduq_a3uJZh9Mpm_tuxA: invalid literal for int() with base 2: ''
4_mduq_a3uJZh9Mpm_tuxA: invalid literal for int() with base 2: ''
tyfMjR6f6vvkM6POJ-vUQQ: invalid literal for int() with base 2: ''
kv2gJs1-r22XqV2lblvVFw: invalid literal for int() with base 2: ''
TYtkYUVAli1N9o7PTH5Jqg: invalid literal for int() with base 2: ''
G7Dp-FLWp7mfowc5pvwIxA: invalid literal for int() with base 2: ''
bha1Us-cEwxmjYRkUpmIPA: invalid literal for int() with base 2: ''
P46mO74cnn